In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
data = pd.read_csv('./output/thrombolysis_choice_benefit_he.csv')
data = data[data['infarction'] ==1]
list(data)

['Unnamed: 0',
 'stroke_team',
 'male',
 'year',
 'arrive_by_ambulance',
 'onset_to_arrival_time',
 'onset_during_sleep',
 'arrival_to_scan_time',
 'onset_to_scan',
 'infarction',
 'stroke_severity',
 'precise_onset_known',
 'prior_disability',
 'any_afib_diagnosis',
 'afib_anticoagulant',
 'age',
 'onset_to_thrombolysis',
 'thrombolysis',
 'thrombectomy',
 'scan_to_thrombolysis_time',
 'discharge_disability',
 'simulated_onset_to_thrombolysis',
 'thrombolysis_choice_probability',
 'thrombolysis_choice',
 'hospital_SHAP',
 'benchmark_decision',
 'untreated_weighted_mrs',
 'untreated_0_to_4',
 'untreated_mrs_0',
 'untreated_mrs_1',
 'untreated_mrs_2',
 'untreated_mrs_3',
 'untreated_mrs_4',
 'untreated_mrs_5',
 'untreated_mrs_6',
 'untreated_utility',
 'treated_weighted_mrs',
 'treated_0_to_4',
 'treated_mrs_0',
 'treated_mrs_1',
 'treated_mrs_2',
 'treated_mrs_3',
 'treated_mrs_4',
 'treated_mrs_5',
 'treated_mrs_6',
 'treated_utility',
 'change_in_weighted_mrs',
 'change_in_mrs_0_to_4

In [10]:
# Limit to no afib
mask = data['afib_anticoagulant'] == 0
data = data[mask]

proportion = 1 - mask.mean()
print (f'Proportion of patients receiving anticoagulants: {proportion:.2f}')

Proportion of patients receiving anticoagulants: 0.13


In [11]:
# Set dataframe dispplay rows to 100
pd.set_option('display.max_rows', 100)

data.head().T

,0,2,4,6,7
Unnamed: 0,0,2,5,7,8
stroke_team,Royal Berkshire Hospital,Royal Berkshire Hospital,Queen Elizabeth Hospital Edgbaston,Wycombe General Hospital,Queen's Medical Centre - Nottingham
male,0,1,0,1,1
year,2017,2017,2016,2021,2020
arrive_by_ambulance,1.0,1.0,1.0,1.0,1.0
onset_to_arrival_time,65.0,190.0,111.0,96.0,138.0
onset_during_sleep,0,0,0,0,0
arrival_to_scan_time,5.0,15.0,39.0,6.0,52.0
onset_to_scan,70.0,205.0,150.0,102.0,190.0
infarction,1.0,1.0,1.0,1.0,1.0


In [12]:
# Set HE columns when dead

mask = data['used_untreated_mrs'] == 6
data.loc[mask, 'untreated_life_expectancy'] = data.loc[mask, 'age']
data.loc[mask, 'untreated_qalys_total'] = 0
data.loc[mask, 'untreated_care_years'] = 0
data.loc[mask, 'untreated_survival_median_years'] = 0

mask = data['used_treated_mrs'] == 6
data.loc[mask, 'treated_life_expectancy'] = data.loc[mask, 'age']
data.loc[mask, 'treated_qalys_total'] = 0
data.loc[mask, 'treated_care_years'] = 0
data.loc[mask, 'treated_survival_median_years'] = 0


In [15]:
# Show descriptive statistics for treated patients
display_cols = [
    'untreated_survival_median_years',
    'treated_survival_median_years',
    'untreated_life_expectancy',
    'treated_life_expectancy',
    'untreated_care_years',
    'treated_care_years',
    'untreated_qalys_total',
    'treated_qalys_total',
    'untreated_life_expectancy',
    'treated_life_expectancy']


mask = data['thrombolysis'] == 1
data.loc[mask][display_cols].describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
untreated_survival_median_years,32289.0,7.61,8.55,0.00,1.24,4.11,10.46,43.23
treated_survival_median_years,32490.0,7.91,8.63,0.00,1.38,4.72,11.82,43.23
untreated_life_expectancy,32289.0,81.99,6.85,38.63,77.22,82.02,87.50,95.40
treated_life_expectancy,32490.0,82.27,6.47,38.63,77.72,82.51,87.50,95.09
untreated_care_years,32289.0,0.28,0.43,0.00,0.07,0.19,0.34,7.58
treated_care_years,32490.0,0.25,0.35,0.00,0.07,0.17,0.33,7.58
untreated_qalys_total,32289.0,5.02,5.44,0.00,0.62,2.88,7.46,22.46
treated_qalys_total,32490.0,5.26,5.49,0.00,0.67,3.79,7.77,22.46
untreated_life_expectancy,32289.0,81.99,6.85,38.63,77.22,82.02,87.50,95.40
treated_life_expectancy,32490.0,82.27,6.47,38.63,77.72,82.51,87.50,95.09


In [17]:
data['benchmark_decision'].mean()

0.49129699571684604

In [21]:
data['improved_outcome'].mean() / data['thrombolysis'].mean()

1.2745143926983384

In [22]:
data['improved_outcome'].mean()

0.49629907147702995